In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import collections
import matplotlib.pyplot as plt
import json

In [2]:
def get_api_key(): #To be called inside of "create_watcher_obj()"
    
    api_file = open("../riot_api_key.txt","r")
    api_key = api_file.read()
    api_file.close()
    return api_key

In [3]:
def create_watcher_obj(): 
    api_key = get_api_key()
    watcher = LolWatcher(api_key)
    return watcher

In [4]:
watcher = create_watcher_obj()

In [5]:
def get_acc_id(region='na1', summoner_name='we and i'): #To be called inside of "get_match_list(x,y)"
     
    watcher_summoner = watcher.summoner.by_name(region,summoner_name)
    return watcher_summoner['accountId']

In [6]:
def get_match_list(region='na1', summoner_name="we and i"):
    
    acc_id = get_acc_id(region, summoner_name)
    my_matches = watcher.match.matchlist_by_account(region, acc_id)
    return my_matches['matches']
    
    

In [7]:
def get_game_id(match): #Use case is TBD
    
    game_id = match['gameId']
    return game_id

In [8]:
def get_match_detail(game_id, region ='na1'):
    
    match_detail = watcher.match.by_id(region, game_id)
    return match_detail

In [9]:
def get_match_timeline(game_id, region='na1'):
    
    timeline = watcher.match.timeline_by_match(region, game_id)
    return timeline

In [10]:
def get_match_frames(timeline):
    
    timeline_frames = timeline['frames']
    return timeline_frames

In [11]:
def get_specific_match_frame(frames, index):

    return frames[index]

In [12]:
def get_match_positions(game_id, region="na1"):
    
    a_dict = collections.defaultdict(list)
    timeline = get_match_timeline(game_id)
    timeline_frames = get_match_frames(timeline)
    
    for frame in timeline_frames:
        get_frame_positions(frame, a_dict)
                  
    return a_dict

In [13]:
def get_frame_positions(frame, frame_dict):
    
    for participant_num in frame['participantFrames']:
            single_participant_frame = frame['participantFrames'][str(participant_num)]
            if ('position' in single_participant_frame.keys()):
                id_pos_pairing = {single_participant_frame['participantId']:single_participant_frame['position']}
                frame_dict[frame['timestamp']].append(id_pos_pairing)
    return frame_dict
    

In [14]:
def get_EXAMPLE_game_id():
    
    match_list = get_match_list()
    match = match_list[0]
    match_game_id = get_game_id(match)
    
    return match_game_id

In [15]:
def jsonify_pos_dict(pos_dict):
    
    json_object = json.dumps(pos_dict, indent = 4)
    return json_object

In [16]:
def output_pos_dict_json(pos_dict, game_id): #To ensure proper formating, only output a_dict and NOT a 'jsonified' a_dict
    
    with open(f'{game_id}.json', 'w') as outfile:
        json.dump(pos_dict, outfile)

In [17]:
def get_coords_from_frame_obj(frame_dict, timestamp): #Will return the coordinates when given a frame_dictionary (a_dict-esque) and an index
    
    coords = []
    for i in range(10):
        x = tuple(frame_dict[timestamp][i][i+1].values())
        coords.append(x)
        
    return coords

In [18]:
def seperate_coords(coords):
    
    xs_t1 = [tup[0] for tup in coords[0:5]]
    ys_t1 = [tup[1] for tup in coords[0:5]]
    xs_t2 = [tup[0] for tup in coords[5:10]]
    ys_t2 = [tup[1] for tup in coords[5:10]]
    
    return xs_t1, ys_t1, xs_t2, ys_t2

In [19]:
def get_EXAMPLE_coords(): #Same functionality as above, just comes with preloaded frame_dictionary
    
    frame_dict = get_match_positions(get_EXAMPLE_game_id())
    index = 0
    coords = []
    for i in range(10):
        x = tuple(frame_dict[index][i][i+1].values())
        coords.append(x)
        
    return coords

In [20]:
def get_game_json(game_id, region='na1'):
    pass
    

In [21]:
#STATUS: CURRENTLY DESIGNING THE MOST ESSENTIAL METHODS FOR MY INTENDED WEBPAGE FUNCTIONALITY WITH AND HOW IT WOULD UTILIZE 
#        REST API. BASICALLY, USING MY DATA STRUCTURE DIAGRAM, I AM WANTING TO PACKAGE THE ESSENTIAL INFORMATION, CREATE A 
#        REST API WITH FLASK, THEN USE MY CLIENT SIDE REACT SCRIPTING THAT WILL READ THE JSON OFF THE REST API. CURRENTLY,
#        JUST DESIGNING THE METHODS THAT WILL PROCESS THE NECCESSARY INFO. HOW I HAVE IT IMAGINED, 

In [22]:
g_id = get_EXAMPLE_game_id()
m_det = get_match_detail(g_id)
m_tim = get_match_timeline(g_id)

In [23]:
print(m_tim['frames'][1].keys())

dict_keys(['participantFrames', 'events', 'timestamp'])


In [32]:
m_det = get_match_detail(g_id)
print(m_det.keys())

dict_keys(['gameId', 'platformId', 'gameCreation', 'gameDuration', 'queueId', 'mapId', 'seasonId', 'gameVersion', 'gameMode', 'gameType', 'teams', 'participants', 'participantIdentities'])


In [44]:
print(m_det['teams'])

[{'teamId': 100, 'win': 'Win', 'firstBlood': True, 'firstTower': True, 'firstInhibitor': False, 'firstBaron': False, 'firstDragon': True, 'firstRiftHerald': True, 'towerKills': 4, 'inhibitorKills': 0, 'baronKills': 0, 'dragonKills': 2, 'vilemawKills': 0, 'riftHeraldKills': 1, 'dominionVictoryScore': 0, 'bans': []}, {'teamId': 200, 'win': 'Fail', 'firstBlood': False, 'firstTower': False, 'firstInhibitor': False, 'firstBaron': False, 'firstDragon': False, 'firstRiftHerald': False, 'towerKills': 0, 'inhibitorKills': 0, 'baronKills': 0, 'dragonKills': 0, 'vilemawKills': 0, 'riftHeraldKills': 0, 'dominionVictoryScore': 0, 'bans': []}]


In [26]:
print('participant frames: ', m_tim['frames'][10]['participantFrames'], '\n \n')
print('events: ', m_tim['frames'][10]['events'], '\n \n')
print('time: ', m_tim['frames'][10]['timestamp'], '\n \n')

participant frames:  {'1': {'participantId': 1, 'position': {'x': 7082, 'y': 9655}, 'currentGold': 1395, 'totalGold': 3885, 'level': 7, 'xp': 3341, 'minionsKilled': 1, 'jungleMinionsKilled': 58, 'dominionScore': 0, 'teamScore': 0}, '2': {'participantId': 2, 'position': {'x': 1486, 'y': 11015}, 'currentGold': 1562, 'totalGold': 2062, 'level': 8, 'xp': 4333, 'minionsKilled': 27, 'jungleMinionsKilled': 0, 'dominionScore': 0, 'teamScore': 0}, '3': {'participantId': 3, 'position': {'x': 7564, 'y': 8185}, 'currentGold': 960, 'totalGold': 3955, 'level': 8, 'xp': 4338, 'minionsKilled': 55, 'jungleMinionsKilled': 0, 'dominionScore': 0, 'teamScore': 0}, '4': {'participantId': 4, 'position': {'x': 11477, 'y': 1594}, 'currentGold': 213, 'totalGold': 2938, 'level': 6, 'xp': 2725, 'minionsKilled': 39, 'jungleMinionsKilled': 0, 'dominionScore': 0, 'teamScore': 0}, '5': {'participantId': 5, 'position': {'x': 8715, 'y': 1117}, 'currentGold': 503, 'totalGold': 2903, 'level': 6, 'xp': 2474, 'minionsKille

In [30]:
for frame in m_tim['frames']:
    for event in frame['events']:
        print(event['type'])

SKILL_LEVEL_UP
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
SKILL_LEVEL_UP
SKILL_LEVEL_UP
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
SKILL_LEVEL_UP
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
SKILL_LEVEL_UP
SKILL_LEVEL_UP
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
SKILL_LEVEL_UP
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
SKILL_LEVEL_UP
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
WARD_PLACED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_PURCHASED
ITEM_DESTROYED
ITEM_DESTROYED
WARD_PLACED
SKILL_LEVEL_UP
ITEM_DESTROYED
ITEM_DESTROYED
WARD_PLACED
WARD_PLACED
ITEM_DESTROYED
SKILL_LEVEL_UP
SKILL_LEVEL_UP
WARD_PLACED
SKILL_LEVEL_UP
WARD_PLACED
WARD_PLACED
WARD_PLACED
WARD_PLACED
WARD_PLACED
WARD_PLACED
ITEM_DESTROYED
WARD_PLACED
WARD_PLACED
WARD_PLACED
WARD_PLACED
SKILL_LEVEL_UP
WARD_PLACED
WARD_PLACED
ITEM_DESTROYED
WARD_PLACED
WARD_PLACED
WARD_PLACED
ITEM_DESTROYED
SKILL_LEVE

In [29]:
#print('events: ', m_tim['frames'][10]['events'], '\n \n')

for frame in m_tim['frames']:
    for event in frame['events']:
        if(event['type'] == 'CHAMPION_KILL'):
            print(event)

{'type': 'CHAMPION_KILL', 'timestamp': 201722, 'position': {'x': 3389, 'y': 11022}, 'killerId': 1, 'victimId': 6, 'assistingParticipantIds': [3]}
{'type': 'CHAMPION_KILL', 'timestamp': 209490, 'position': {'x': 13506, 'y': 3465}, 'killerId': 4, 'victimId': 10, 'assistingParticipantIds': [5]}
{'type': 'CHAMPION_KILL', 'timestamp': 212827, 'position': {'x': 13977, 'y': 4366}, 'killerId': 5, 'victimId': 8, 'assistingParticipantIds': [4]}
{'type': 'CHAMPION_KILL', 'timestamp': 249839, 'position': {'x': 8340, 'y': 7992}, 'killerId': 1, 'victimId': 7, 'assistingParticipantIds': [3]}
{'type': 'CHAMPION_KILL', 'timestamp': 275879, 'position': {'x': 8861, 'y': 7695}, 'killerId': 3, 'victimId': 6, 'assistingParticipantIds': [1]}
{'type': 'CHAMPION_KILL', 'timestamp': 405202, 'position': {'x': 13352, 'y': 3112}, 'killerId': 3, 'victimId': 10, 'assistingParticipantIds': [1, 4, 5]}
{'type': 'CHAMPION_KILL', 'timestamp': 467574, 'position': {'x': 7095, 'y': 7905}, 'killerId': 3, 'victimId': 7, 'assi

<img src="../reference images/match_history_keys.jpg">

> Note: every match history object has 4 keys. The "get_acc_matches(x,y)" function will only return values corresponding to the 'matches' key.

<img src="../reference images/match_example.jpg">

> Note: every individual match, from a match_list object, has these parameters.

<img src="../reference images/timeline_example.jpg"> 

> Every timeline object has the above keys

<img src="../reference images/timeline_frames_example.jpg"> 

> The timeline's "frames" component does not have any keys because timeline['frames'] is a list data structure

<img src="../reference images/timeline_frames_working_example.jpg">

> Each individual frame, however, is a dictionary object and therefore contains the above keys.

<img src="../reference images/participant_frames_example.jpg">

> Every "participantFrames" object contains information pertaining to each summoner during that specific frame. 

<img src="../reference images/single_participant_frame_example.jpg">

> This is the type of information stored in a single participant's frame. Notice how it is contained in a dictionary, so every parameter stored here can be used as a dictionary key.

<img src="../reference images/frame_intervals_example.jpg">

> Note, when trying to access the different moments of a game, be aware of the fact that each game is different in length, so they will inherently have different quantities of frames. But also, the API data does not capture information at equidistant frame intervals. Probably need to use a ".keys()" addition if you want to access a specific frame that is return from "get_match_frames(x,y).

<img src="../reference images/pos_structure_example.jpg">

> Here is an example of how you would access each layer of the a_dict data structure that is returned by "get_match_positions".
            
    

<img src="../reference images/size_of_map_1.jpg"> 

> For reference, it is estimated that the size of the League of Legends map is about 16,000 'units'. I don't exactly know what that unit means, but I am going to use these parameters to set the bounardies for the matplot map. Additionally, I am operating under the assumption that the League of Legends map is a square.

<img src="../reference images/timestamp_issue.jpg">

> It is probably a good idea to have a seperate handler/data organizer for the match's events. This is because your current data structure uses the timestamps as keys for the various states of the game. However, due to the nature of events, they do not happen syncronously at exact timestamp. Instead, each events kind of has a unique timestamp that pertains to that specific event.

<img src="../reference images/riot api data structure.png">

<img src="../reference images/data_structure_1.jpg">

> Note: this is the updated data structure. Note how each match not only contains timestamp information, but also game summary and events are contained within each match as well. 

<img src="../reference images/match_detail_keys.jpg">

> Each match detail object contains the above keys

<img src="../reference images/last_frame_timeline.jpg"> 

> Note: the very last frame of every game's timeline does not contain the 'position' field, so you need to include a flow mechanism to handle the last frame of every game.

<img src="../reference images/data_struc_events.jpg">

> Note, this is the current setup for the 'events' side of your game data structure. Each event is accessed with its own key. The key is just sequential integers. Additionally, the events are stored in chronlogical order. I didn't use the timestamp as the key for each event simply due to the lack of uniformity in the timestamps and how they are randomly dispersed throughout the game.

<img src="../reference images/constants_doc.jpg">

> Note, riot games has documentation of all of their constants. For example, above are some of the values for mapId. And these constants are all available as json webpages. The link for all of their constants documentation is here: http://static.developer.riotgames.com/docs/lol/maps.json

In [25]:
#On the to do list: download every champion portrait from this link: https://lol.fandom.com/wiki/Category:Champion_Square_Images
# THEN, change each visual indicator on the map to be the champion portrait. There might be some scaling things.